# **Tratamento de Dados**

A função deste arquivo é integralmente para o tratamento do conjunto de dados a ser processado.

Para isso utilizaremos a biblioteca Pandas para a extração e tratamento dos dados.

In [1]:
import pandas as pd

## **Limpeza de Dados**

Começaremos extraindo e visualização nossos dados.

In [2]:
dados = pd.read_csv('./Dataset/Raw/OnlineRetail.csv', encoding='iso-8859-1')

dados.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


De início precisamo saber como estão nossos dados, se temos dados faltantes, duplicados ou com tipagem errada, além do mais algumas colunas não servem para modelos de Machine Learning, devemos indetificá-las e retirá-las de nosso dados.

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Vemos que temos três colunas com problemas, sendo elas `Description`, `InvoiceDate`,  `CustomerID`. A pedido do cliente, separaremos em dois conjuntos de dados para o faturamento e outro para clientes *churn*, excluindo a coluna `Description` de ambas os conjuntos, pois não será necessário e trataremos a coluna `CustomerID` apenas para os dados *churn*, enquanto para o faturamento apenas excluiremos. Para a coluna `InvoiceDate`, precisamo garantir que ela esteja no formato de data para conseguirmos ter uma precisão na ordem das vendas.

Outros problemas que podemos ver é com relação ao tipo da coluna `CustomerID` que está do tipagem errada, precisamos garantir que esteja no tipo certo para evitar perca de informação. Além disso excluiremos as colunas `StockCode` e `Country` que não farão parte de nossas análises.

Mas antes de tudo isso devemos ver quantos dados duplicados temos para não enviesas nossas análises.

In [4]:
dados.duplicated().sum()

np.int64(5268)

In [5]:
dados = dados.drop_duplicates()

In [6]:
dados.duplicated().sum()

np.int64(0)

Vimos que tinhamos 5268 dados duplicados, isso poderia afetar principalmente o faturamento da loja em cada ano, além de afetar nosso modelo de previsão. Continuaremos para a exclusão de colunas inúteis, a separação e tratamento de dados nulos e formato dos nossos dados.

In [7]:
dados = dados.drop(['StockCode', 'Country'], axis=1)

In [8]:
dados['InvoiceDate'] = pd.to_datetime(dados['InvoiceDate'])
dados['InvoiceDate'] = pd.to_datetime(dados['InvoiceDate'].dt.date)

In [9]:
df_faturamento = dados.drop(['CustomerID', 'InvoiceNo', 'Description'], axis=1).copy()
df_faturamento.head()

,Quantity,InvoiceDate,UnitPrice
0,6,2010-12-01,2.55
1,6,2010-12-01,3.39
2,8,2010-12-01,2.75
3,6,2010-12-01,3.39
4,6,2010-12-01,3.39


In [10]:
df_churn = dados.dropna().copy()
df_churn.head()

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0
1,536365,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0


Com isso, precisamos apenas atualizar a tipagem da coluna `CustomerID` de nosso dados *churn*.

In [11]:
df_churn['CustomerID'] = df_churn['CustomerID'].astype(int)

In [12]:
df_churn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401604 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   Description  401604 non-null  object        
 2   Quantity     401604 non-null  int64         
 3   InvoiceDate  401604 non-null  datetime64[ns]
 4   UnitPrice    401604 non-null  float64       
 5   CustomerID   401604 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 21.4+ MB


In [13]:
df_faturamento.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536641 entries, 0 to 541908
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Quantity     536641 non-null  int64         
 1   InvoiceDate  536641 non-null  datetime64[ns]
 2   UnitPrice    536641 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 16.4 MB


## **Regras de Negócio**

Além do primeiro tratamento de nossos dados, nosso cliente apontou características importantes para podemos definir um cliente como *churn* ou não. Sendo elas:

- Deixa de comprar a mais de 60 dias, para possíveis *churn's* e 90 dias para *churn's*;

- Se um cliente tem uma taxa de devolução superior a 40%;

- Compra depois de mais de 45 dias e gasta menos de 500 reais mensal. Um grande *churn* em potêncial.

- Se o tempo médio entre compras do cliente é de 15 dias, e ele já está há 40 dias sem comprar, ele é um forte candidato ao *churn*.

- Se o cliente comprou apenas 1 vez, um grande *churn* em potêncial.

Para operações matemática utilizaremos a biblioteca Numpy.

In [14]:
import numpy as np

Visto que temos muitos problemas, vamos focar apenas nos dados do *df_churn*, visto que o *df_faturamento* vai focar em outros aspectos de nossa análise. Com isso atribuiremos colunas como `PossivelChurn`, `TaxaDevolucao`, `GastoMedio`, `GastoTotal`, `GastoMensal`, `MediaDiasCompra`, `TempoUltimoDiaCompra` e `Churn`. Com isso podemos investigar o coportamento de cada cliente, se ele é um cliente que devolve muito, compra muito, se ele compra mais pelo cupom e se ele virará um *churn* ou não, dado que apenas nossas regras de negócio não são suficiente para dizer o que vira churn ou não.

E Para começar, vamos calcular o valor total gasto por objeto e juntar tudo com o *ID* e o número da fatura da compra.

In [15]:
df_churn['Preco_Total'] = df_churn['UnitPrice'] * df_churn['Quantity']
df_churn.head()

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Preco_Total
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,15.30
1,536365,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,20.34
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,22.00
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,20.34
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,20.34


In [16]:
df_cliente_fatura = df_churn[['InvoiceNo', 'InvoiceDate', 'CustomerID', 'Preco_Total']].groupby(['CustomerID','InvoiceNo', 'InvoiceDate']).sum().copy()

df_cliente_fatura = df_cliente_fatura.reset_index()
df_cliente_fatura.head()

,CustomerID,InvoiceNo,InvoiceDate,Preco_Total
0,12346,541431,2011-01-18,77183.60
1,12346,C541433,2011-01-18,-77183.60
2,12347,537626,2010-12-07,711.79
3,12347,542237,2011-01-26,475.39
4,12347,549222,2011-04-07,636.25


## **Média e Último Dias de Compra**

Vamos começar calculando a frequência média de comprase o último dia de compra. Neste caso, vamos calcular o quão frenquente cada pessoa costuma ir na loja e quando foi a última vez que ela foi na loja em comparação ao último dia registrado.

In [17]:
df_cliente_fatura = df_cliente_fatura.sort_values(by=['CustomerID', 'InvoiceDate'])
df_cliente_fatura['UltimaCompraDias'] = df_cliente_fatura.groupby('CustomerID')['InvoiceDate'].diff()

df_cliente_fatura['UltimaCompraDias'] = df_cliente_fatura['UltimaCompraDias'].dt.days.fillna(0)
df_cliente_fatura['UltimaCompraDias'] = df_cliente_fatura['UltimaCompraDias'].astype(int)

df_cliente_fatura.head()

,CustomerID,InvoiceNo,InvoiceDate,Preco_Total,UltimaCompraDias
0,12346,541431,2011-01-18,77183.60,0
1,12346,C541433,2011-01-18,-77183.60,0
2,12347,537626,2010-12-07,711.79,0
3,12347,542237,2011-01-26,475.39,50
4,12347,549222,2011-04-07,636.25,71


In [18]:
frequencia_compras = df_cliente_fatura[['CustomerID', 'UltimaCompraDias']].copy()
frequencia_compras.columns = ['CustomerID', 'MediaDiasCompra']

frequencia_compras = frequencia_compras.groupby('CustomerID').mean()
frequencia_compras['MediaDiasCompra'] = np.ceil(frequencia_compras['MediaDiasCompra'])
frequencia_compras['MediaDiasCompra'] = frequencia_compras['MediaDiasCompra'].astype(int)
frequencia_compras = frequencia_compras.reset_index()

In [19]:
dias_pos_compra = (pd.to_datetime('2011-12-09') - df_cliente_fatura['InvoiceDate'])
df_cliente_fatura['TempoUltimoDiaCompra'] = dias_pos_compra.dt.days

In [24]:
tempo_ultimo_dia_compra = df_cliente_fatura[['CustomerID', 'TempoUltimoDiaCompra']].copy()

tempo_ultimo_dia_compra = tempo_ultimo_dia_compra.groupby('CustomerID').agg('min')
tempo_ultimo_dia_compra = tempo_ultimo_dia_compra.reset_index()

In [111]:
df_cliente_fatura = df_cliente_fatura.drop(['InvoiceDate', 'UltimaCompraDias', 'TempoUltimoDiaCompra'], axis=1)

Com isso excluiremos a coluna `UltimaCompraDias` para um conjunto de dados mais limpo e não nos será mais útil.

## **Taxa de Devolução**

Seguindo isso, vamos calcular a taxa de devolução dos clientes.

In [112]:
df_cliente_fatura['Comprou/Devolveu'] = [1 if x > 0 else -1 for x in df_cliente_fatura['Preco_Total']]
df_cliente_fatura['Operacao'] = 1
df_cliente_fatura.head()

,CustomerID,InvoiceNo,Preco_Total,Comprou/Devolveu,Operacao
0,12346,541431,77183.60,1,1
1,12346,C541433,-77183.60,-1,1
2,12347,537626,711.79,1,1
3,12347,542237,475.39,1,1
4,12347,549222,636.25,1,1


In [113]:
taxa_devolucao = df_cliente_fatura[['CustomerID', 'Comprou/Devolveu', 'Operacao']].groupby(['CustomerID']).sum().copy()

for linha in range(len(taxa_devolucao['Operacao'])):
    if taxa_devolucao.iloc[linha, 0] != taxa_devolucao.iloc[linha, 1]:
        retirar = (taxa_devolucao.iloc[linha, 1] - taxa_devolucao.iloc[linha, 0]) / 2
        taxa_devolucao.iloc[linha, 1] -= retirar

In [114]:
taxa_devolucao['PorcentagemCompra'] = taxa_devolucao['Comprou/Devolveu'] / taxa_devolucao['Operacao']

taxa_devolucao['TaxaDevolucao'] = np.round(1 - taxa_devolucao['PorcentagemCompra'],2)
taxa_devolucao = taxa_devolucao.drop(['Comprou/Devolveu', 'Operacao', 'PorcentagemCompra'], axis = 1)
taxa_devolucao = taxa_devolucao.reset_index()

In [115]:
df_cliente_fatura = df_cliente_fatura.drop(['InvoiceNo', 'Comprou/Devolveu', 'Operacao'], axis = 1)

## **Gasto Total, Médio e Mensal**

Agora, após termos separado mais uma vez e retirado as linhas que não usaremos mais, vamos para os últimos cálculos, sendo eles a média gasta, o valor total gasto e o gasto mensal.

In [116]:
gasto_total = df_cliente_fatura[['CustomerID', 'Preco_Total']].groupby('CustomerID').sum().copy()
gasto_total.columns = ['GastoTotal']
gasto_total['GastoTotal'] = np.round(gasto_total['GastoTotal'], 2)
gasto_total = gasto_total.reset_index()

Como estamo avaliando dados de apenas 2011, então o gasto mensal será o total por 12.

In [117]:
gasto_mensal = gasto_total.copy()
gasto_mensal['GastoMensal'] = np.round(gasto_mensal['GastoTotal'] / 12, 2)
gasto_mensal = gasto_mensal.drop('GastoTotal', axis=1)

In [118]:
gasto_medio = df_cliente_fatura[['CustomerID', 'Preco_Total']].groupby('CustomerID').mean().copy()
gasto_medio.columns = ['GastoMedio']
gasto_medio['GastoMedio'] = np.round(gasto_medio['GastoMedio'], 2)
gasto_medio = gasto_medio.reset_index()

In [119]:
df_cliente_fatura = df_cliente_fatura.drop('Preco_Total', axis = 1)
df_cliente_fatura = df_cliente_fatura.groupby(['CustomerID']).sum().reset_index()

In [120]:
df_cliente_fatura = df_cliente_fatura.merge(gasto_total, on='CustomerID')
df_cliente_fatura = df_cliente_fatura.merge(gasto_mensal, on='CustomerID')
df_cliente_fatura = df_cliente_fatura.merge(gasto_medio, on='CustomerID')
df_cliente_fatura = df_cliente_fatura.merge(taxa_devolucao, on='CustomerID')
df_cliente_fatura = df_cliente_fatura.merge(frequencia_compras, on='CustomerID')
df_cliente_fatura = df_cliente_fatura.merge(tempo_ultimo_dia_compra, on='CustomerID')

df_cliente_fatura.head()

,CustomerID,GastoTotal,GastoMensal,GastoMedio,TaxaDevolucao,MediaDiasCompra,TempoUltimoDiaCompra
0,12346,0.00,0.00,0.00,1.0,0,325
1,12347,4310.00,359.17,615.71,0.0,53,2
2,12348,1797.24,149.77,449.31,0.0,71,75
3,12349,1757.55,146.46,1757.55,0.0,0,18
4,12350,334.40,27.87,334.40,0.0,0,310


In [121]:
df_cliente_fatura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4372 entries, 0 to 4371
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerID            4372 non-null   int64  
 1   GastoTotal            4372 non-null   float64
 2   GastoMensal           4372 non-null   float64
 3   GastoMedio            4372 non-null   float64
 4   TaxaDevolucao         4372 non-null   float64
 5   MediaDiasCompra       4372 non-null   int64  
 6   TempoUltimoDiaCompra  4372 non-null   int64  
dtypes: float64(4), int64(3)
memory usage: 239.2 KB


## **Aplicação para Churn's**

Para a aplicação das regras de *churn*, vamos verificar primeiro se os dados estão condizendo com nossa proposta de 90 dias para ser considerado *churn*, com isso, vamos ver a média de quanto tempo se passou desde de a última compra com a data do último registro dos dados e a média de frequência de compras das pessoas.

In [122]:
df_cliente_fatura['TempoUltimoDiaCompra'].mean()

np.float64(91.58119853613907)

In [123]:
df_cliente_fatura['MediaDiasCompra'].mean()

np.float64(28.184583714547117)

In [124]:
df_cliente_fatura['GastoTotal'].mean()

np.float64(1893.5314318389755)

In [125]:
df_cliente_fatura['GastoMensal'].mean()

np.float64(157.7942680695334)

E como podemos ver há um valor discrepante para o último de de compra com a frequência de compra, isso confirma que a loja está tendo sim problema na retenção de clientes. Com isso podemos avançar e aplicar nossas regras para *churn's*.

Apesar de bons resultados, vemos que o gasto mensal do ano de 2011, foi de 157 mensais, o que não correspondem a um de nossos filtros, para isso iremos adaptá-lo para nosso dados reais.

In [126]:
condicoes = [
    (df_cliente_fatura['TempoUltimoDiaCompra'] > 60),
    (df_cliente_fatura['MediaDiasCompra'] <= 15) & (df_cliente_fatura['TempoUltimoDiaCompra'] >= 40),
    (df_cliente_fatura['TempoUltimoDiaCompra'] > 45) & (df_cliente_fatura['GastoMedio'] < 157),
    (df_cliente_fatura['GastoMedio'] == df_cliente_fatura['GastoTotal'])
]

escolhas = [1, 1, 1, 1]

df_cliente_fatura['PossivelChurn'] = np.select(condicoes, escolhas, default=0)
df_cliente_fatura.head()

,CustomerID,GastoTotal,GastoMensal,GastoMedio,TaxaDevolucao,MediaDiasCompra,TempoUltimoDiaCompra,PossivelChurn
0,12346,0.00,0.00,0.00,1.0,0,325,1
1,12347,4310.00,359.17,615.71,0.0,53,2,0
2,12348,1797.24,149.77,449.31,0.0,71,75,1
3,12349,1757.55,146.46,1757.55,0.0,0,18,1
4,12350,334.40,27.87,334.40,0.0,0,310,1


In [127]:
condicoes_churn = [
    (df_cliente_fatura['TempoUltimoDiaCompra'] > 90),
    (df_cliente_fatura['TaxaDevolucao'] > 0.4),
]

escolhas_churn = [1, 1]

df_cliente_fatura['Churn'] = np.select(condicoes_churn, escolhas_churn, default=0)
df_cliente_fatura.head()

,CustomerID,GastoTotal,GastoMensal,GastoMedio,TaxaDevolucao,MediaDiasCompra,TempoUltimoDiaCompra,PossivelChurn,Churn
0,12346,0.00,0.00,0.00,1.0,0,325,1,1
1,12347,4310.00,359.17,615.71,0.0,53,2,0,0
2,12348,1797.24,149.77,449.31,0.0,71,75,1,0
3,12349,1757.55,146.46,1757.55,0.0,0,18,1,0
4,12350,334.40,27.87,334.40,0.0,0,310,1,1


In [128]:
(df_cliente_fatura['PossivelChurn'] != df_cliente_fatura['Churn']).sum()

np.int64(1206)

In [129]:
(df_cliente_fatura['Churn'] == 1).sum()

np.int64(1940)

In [130]:
(df_cliente_fatura['PossivelChurn'] == 1).sum()

np.int64(2422)

Após aplicarmos as regras para *churn's* fizemos uma comparação para ver se avaliar se nossas colunas `PossivelChurn` e `Churn` são iguais. Verificameos que de 4372, ao todo 1206 são diferentes, o que é um resultado satisfatório para não enviesarmos nossos dados para nosso modelo de Machine Learning. Além disso, vemos que quase todos os nossos dados são possíveis *churn's* para nossa base, sendo que 1940 são de fato *churn's* e 2422 são possíveis *churn's*, o que nos leva uma grande preocupação sobre quais serão os próximos *churn's*.

Vemos portanto que nossas conjunto de dados estão devidamente convertidos e tratados, podemos salvar e utilizar cada um deles para seus respectivos propósitos.

In [131]:
df_faturamento.to_csv('./Dataset/Tratado/faturamento.csv', index=False)
df_churn.to_csv('./Dataset/Tratado/dados_gerais.csv', index=False)
df_cliente_fatura.to_csv('./Dataset/Tratado/clientes_churns.csv', index=False)